##CONDA

In [1]:
# download the CONDA dataset
!mkdir datasets
!mkdir datasets/conda
!wget -P datasets/conda/ -c 'https://raw.githubusercontent.com/usydnlp/CONDA/main/data/CONDA_train.csv'
!wget -P datasets/conda/ -c 'https://raw.githubusercontent.com/usydnlp/CONDA/main/data/CONDA_valid.csv'
!wget -P datasets/conda/ -c 'https://raw.githubusercontent.com/usydnlp/CONDA/main/data/CONDA_test.csv'

--2021-08-24 22:12:37--  https://raw.githubusercontent.com/usydnlp/CONDA/main/data/CONDA_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2856782 (2.7M) [text/plain]
Saving to: ‘datasets/conda/CONDA_train.csv’

CONDA_train.csv     100%[===================>]   2.72M  --.-KB/s    in 0.02s   

2021-08-24 22:12:38 (145 MB/s) - ‘datasets/conda/CONDA_train.csv’ saved [2856782/2856782]

--2021-08-24 22:12:38--  https://raw.githubusercontent.com/usydnlp/CONDA/main/data/CONDA_valid.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Lengt

In [2]:
import pandas as pd

# import the dataset into pandas dataframe
conda_train = pd.read_csv('datasets/conda/CONDA_train.csv')
conda_val = pd.read_csv('datasets/conda/CONDA_valid.csv')
conda_test = pd.read_csv('datasets/conda/CONDA_test.csv')
conda_train.head(6)

,Id,matchId,conversationId,utterance,chatTime,playerSlot,playerId,intentClass,slotClasses,slotTokens
0,11263,697,3193,wow!,76,0,ANTS IN MY EYES JOHNSON,O,O,"wow (O),"
1,13741,843,3809,WTF,1563,5,M.k,O,T,"WTF (T),"
2,22125,1412,6199,wpe wpe,2853,1,Acqua Ragia,O,O O,"wpe (O), wpe (O),"
3,6453,439,1875,hahaha,1038,0,juicebox,O,O,"hahaha (O),"
4,9644,601,2713,wtf,1661,5,KAIST.Shadows,O,T,"wtf (T),"
5,6208,433,1836,i cant [SEPA] play [SEPA] with 4 trash,1674,6,Vintage <3,E,P O SEPA O SEPA O O O,"i (P), cant (O), [SEPA] (SEPA), play (O), [SEP..."


In [3]:
# check for any null items
print("train set:")
print(conda_train.isnull().sum())
print("\nval set:")
print(conda_val.isnull().sum())
print("\ntest set:")
print(conda_test.isnull().sum())

train set:
Id                  0
matchId             0
conversationId      0
utterance           7
chatTime            0
playerSlot          0
playerId            8
intentClass         0
slotClasses       834
slotTokens        834
dtype: int64

val set:
Id                  0
matchId             0
conversationId      0
utterance           1
chatTime            0
playerSlot          0
playerId            1
intentClass         0
slotClasses       268
slotTokens        268
dtype: int64

test set:
Id                0
matchId           0
conversationId    0
utterance         1
chatTime          0
playerSlot        0
playerId          3
dtype: int64


In [4]:
# remove rows with nan values
conda_train.dropna(inplace=True)
conda_val.dropna(inplace=True)
conda_test.dropna(inplace=True)

In [5]:
# create a function to convert the conda dataset into jointBERT input
def create_input_file(data, column, file_name):
    # create a list of the slot text and tokens
    texts = list(data[column])

    # create a list of slot labels
    slot_label = ['(C)','(D)','(O)','(P)','(S)','(T)','(SEPA)',
                  '(C),','(D),','(O),','(P),','(S),','(T),','(SEPA),']

    # extract the text from the slot text and tokens
    all_new_text = []
    for text in texts:
      try: split_text = text.split()
      except: print(text) 
      new_text = []
      for stext in split_text:
        if stext not in slot_label:
          new_text.append(stext)
      join_new_text = " ".join(new_text)
      all_new_text.append(join_new_text)

    # write the text into seq.in file for text input to jointBERT model
    with open(file_name, 'w') as f:
        for item in all_new_text:
            f.write("%s\n" % item)

In [7]:
# create input files for the train set
!mkdir datasets/conda/train
create_input_file(conda_train, 'slotTokens', 'datasets/conda/train/seq.in')
create_input_file(conda_train, 'slotClasses', 'datasets/conda/train/seq.out')
create_input_file(conda_train, 'intentClass', 'datasets/conda/train/label')

In [8]:
# create input files for the val set
!mkdir datasets/conda/val
create_input_file(conda_val, 'slotTokens', 'datasets/conda/val/seq.in')
create_input_file(conda_val, 'slotClasses', 'datasets/conda/val/seq.out')
create_input_file(conda_val, 'intentClass', 'datasets/conda/val/label')

In [9]:
# create input files for the test set (there is no slot/intent labels for the test set)
!mkdir datasets/conda/test
create_input_file(conda_test, 'utterance', 'datasets/conda/test/sample_pred_in.txt')

In [6]:
# create a function to convert the conda dataset into jointBERT input
def create_input_file2(data, column):
    # create a list of the slot text and tokens
    texts = list(data[column])

    # create a list of slot labels
    slot_label = ['(C)','(D)','(O)','(P)','(S)','(T)','(SEPA)',
                  '(C),','(D),','(O),','(P),','(S),','(T),','(SEPA),']

    # extract the text from the slot text and tokens
    all_new_text = []
    for text in texts:
      try: split_text = text.split()
      except: print(text) 
      new_text = []
      for stext in split_text:
        if stext not in slot_label:
          new_text.append(stext)
      join_new_text = " ".join(new_text)
      all_new_text.append(join_new_text)
    
    return all_new_text

In [40]:
train_text = create_input_file2(conda_train, 'slotTokens')
train_text = ['BOS ' + text + ' EOS\t' for text in train_text]
train_text[:5]

['BOS wow EOS\t',
 'BOS WTF EOS\t',
 'BOS wpe wpe EOS\t',
 'BOS hahaha EOS\t',
 'BOS wtf EOS\t']

In [41]:
train_slot = create_input_file2(conda_train, 'slotClasses')
train_slot[:5]

['O', 'T', 'O O', 'O', 'T']

In [42]:
train_label = create_input_file2(conda_train, 'intentClass')
train_label[:5]

['O', 'O', 'O', 'O', 'O']

In [43]:
result = list(zip(train_text, train_slot))
result = ["".join(text) for text in result]
result[:5]

['BOS wow EOS\tO',
 'BOS WTF EOS\tT',
 'BOS wpe wpe EOS\tO O',
 'BOS hahaha EOS\tO',
 'BOS wtf EOS\tT']

In [44]:
result_w_intent = list(zip(result, train_label))
result_w_intent = [" ".join(text) for text in result_w_intent]
result_w_intent[:5]

['BOS wow EOS\tO O',
 'BOS WTF EOS\tT O',
 'BOS wpe wpe EOS\tO O O',
 'BOS hahaha EOS\tO O',
 'BOS wtf EOS\tT O']

In [45]:
with open('datasets/conda/conda.train.w-intent.iob', 'w') as f:
  for item in result_w_intent:
    f.write("%s\n" % item)

with open('datasets/conda/conda.train.iob', 'w') as f:
  for item in result:
    f.write("%s\n" % item)

In [46]:
val_text = create_input_file2(conda_val, 'slotTokens')
val_text = ['BOS ' + text + ' EOS\t' for text in val_text]
val_slot = create_input_file2(conda_val, 'slotClasses')
val_label = create_input_file2(conda_val, 'intentClass')

result = list(zip(val_text, val_slot))
result = ["".join(text) for text in result]
result_w_intent = list(zip(result, val_label))
result_w_intent = [" ".join(text) for text in result_w_intent]

with open('datasets/conda/conda.dev.w-intent.iob', 'w') as f:
  for item in result_w_intent:
    f.write("%s\n" % item)

with open('datasets/conda/conda.dev.iob', 'w') as f:
  for item in result:
    f.write("%s\n" % item)

##LOW

In [18]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# download the low dataset
!mkdir datasets/low

# train
id = '1-CnkX1f_OiAYq5LLc7W_Bxs2hO3aNBgy'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('datasets/low/train.csv')

# train label
id = '1-C8mhh9wbiHzAKodKKH3FKUOydqfAv9K'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('datasets/low/train_label.csv')

# val
id = '1-Cc6X5tMm6FpH1PsQWv_pZYqDbZ_GpCe'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('datasets/low/val.csv')

# val label
id = '1-1PuElSAl1dcJJxdB1cs2KvNf35NnaSc'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('datasets/low/val_label.csv')

# test
id = '1-Cl6P0nOJRErw0BmzQ_OLWPTcbCczFMo'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('datasets/low/test.csv')

# test label
id = '1-40KJhMjrv7Yxkk-A6NYEMkbPl9sjv_4'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('datasets/low/test_label.csv')

In [48]:
import pandas as pd

# import the dataset into pandas dataframe
train_text = pd.read_csv('datasets/low/train.csv', index_col=0)
train_label = pd.read_csv('datasets/low/train_label.csv', index_col=0)
val_text = pd.read_csv('datasets/low/val.csv', index_col=0)
val_label = pd.read_csv('datasets/low/val_label.csv', index_col=0)
test_text = pd.read_csv('datasets/low/test.csv', index_col=0)
test_label = pd.read_csv('datasets/low/test_label.csv', index_col=0)

In [49]:
train_text.head()

,0,1,2,3,4,5,6,7,8,9,10,Unnamed: 12
13430,89498,nothinh come :D,enemy,00:07:29,8,2.0,0.0,Negative Attitude,1423.0,Nasus,O O O,NaN
28658,191675,"malph, would you gank bot?",enemy,00:08:58,4,1.0,0.0,Offensive Language,1973.0,Caitlyn,O O P S O,NaN
20418,135440,no flame just rage,offender,00:07:55,7,2.0,1.0,Verbal Abuse,1666.0,Gragas,O O O O,NaN
19964,133016,i do,enemy,00:35:16,7,2.0,0.0,Negative Attitude,1654.0,Warwick,P O,NaN
4446,22364,no mana :/,enemy,00:07:08,6,0.0,2.0,Negative Attitude,107.0,Ezreal,O S O,NaN


In [50]:
train_label.head()

,Label
13430,O
28658,O
20418,A
19964,O
4446,O


In [51]:
print(train_text.shape)
print(train_label.shape)

(29399, 12)
(29399, 1)


In [52]:
# concat the text and the label files
train_data = pd.concat([train_text,train_label],axis=1,join='inner')
val_data = pd.concat([val_text,val_label],axis=1,join='inner')
test_data = pd.concat([test_text,test_label],axis=1,join='inner')

# drop the 'Unnamed: 12' column
train_data.drop(['Unnamed: 12'], axis=1, inplace=True)
val_data.drop(['Unnamed: 12'], axis=1, inplace=True)
test_data.drop(['Unnamed: 12'], axis=1, inplace=True)
print(train_data.shape)
print(val_data.shape)
print(test_data.shape)

(29399, 12)
(3267, 12)
(3630, 12)


In [53]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,10,Label
13430,89498,nothinh come :D,enemy,00:07:29,8,2.0,0.0,Negative Attitude,1423.0,Nasus,O O O,O
28658,191675,"malph, would you gank bot?",enemy,00:08:58,4,1.0,0.0,Offensive Language,1973.0,Caitlyn,O O P S O,O
20418,135440,no flame just rage,offender,00:07:55,7,2.0,1.0,Verbal Abuse,1666.0,Gragas,O O O O,A
19964,133016,i do,enemy,00:35:16,7,2.0,0.0,Negative Attitude,1654.0,Warwick,P O,O
4446,22364,no mana :/,enemy,00:07:08,6,0.0,2.0,Negative Attitude,107.0,Ezreal,O S O,O


In [54]:
# check for any null items
print("train set:")
print(train_data.isnull().sum())
print("\nval set:")
print(val_data.isnull().sum())
print("\ntest set:")
print(test_data.isnull().sum())

train set:
0         0
1        14
2         0
3         0
4         0
5         1
6         1
7         1
8         1
9         1
10        1
Label     1
dtype: int64

val set:
0        0
1        5
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
Label    0
dtype: int64

test set:
0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
Label    0
dtype: int64


In [55]:
# remove rows with nan values
train_data.dropna(inplace=True)
val_data.dropna(inplace=True)
print("train set:")
print(train_data.isnull().sum())
print("\nval set:")
print(val_data.isnull().sum())

train set:
0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
Label    0
dtype: int64

val set:
0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
Label    0
dtype: int64


In [56]:
# delete texts that contain '#ERROR!'
train_data.drop(train_data['1'][train_data['1']=='#ERROR!'].index, inplace=True)
val_data.drop(val_data['1'][val_data['1']=='#ERROR!'].index, inplace=True)
test_data.drop(test_data['1'][test_data['1']=='#ERROR!'].index, inplace=True)

In [57]:
# check the intent labels
print(set(train_data['Label']))
print(set(val_data['Label']))
print(set(test_data['Label']))

{'I ', 'E', 'I', 'O', 'A'}
{'T', 'E', 'I', 'O', 'A'}
{'A', 'E', 'O', 'I'}


In [58]:
# remove the val data with 'T' intent label
val_data.drop(val_data['Label'][val_data['Label']=='T'].index, inplace=True)

In [59]:
# 'I' and 'I ' intent labels in train set should be the same as 'I'
labels = [label.strip().upper() for label in list(train_data['Label'])]
train_data['Label']=labels
print(set(train_data['Label']))
print(set(val_data['Label']))
print(set(test_data['Label']))

{'A', 'E', 'O', 'I'}
{'A', 'E', 'O', 'I'}
{'A', 'E', 'O', 'I'}


In [60]:
print(train_data.shape)
print(val_data.shape)
print(test_data.shape)

(29370, 12)
(3259, 12)
(3630, 12)


In [61]:
# create a function to remove rows in which the length of the text != the length of the slot tokens
def remove_error(data):
    text_slot = list(zip(list(data['1']),list(data['10'])))

    to_remove = []
    for i, text in enumerate(text_slot):
      if len(str(text[0]).split())!=len(text[1].split()):
        to_remove.append(i)
        print("index:",i)
        print("text:",text[0])
        print("slot label:",text[1])

    to_remove_index = [data.iloc[[n]].index[0] for n in to_remove]
    data.drop(to_remove_index, inplace=True)
    print(data.shape)

In [62]:
# remove rows in train set 
remove_error(train_data)

index: 3301
text:  ''''''''''''' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'p dfd
slot label: O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O
index: 11968
text:  ' '
slot label: O O O
index: 12918
text:  '
slot label: O O
index: 13366
text:  '
slot label: O O
index: 15245
text:  what¿?
slot label: O O
index: 16196
text: I
slot label: S O
index: 18916
text: I
slot label: S L O O
index: 20506
text: O
slot label: S S
index: 20547
text:  ' ' ' ' ' ' '
slot label: O O O O O O O O
index: 26000
text: I
slot label: O P O L
index: 27479
text: E
slot label: O S L S
index: 27966
text: I
slot label: O O O O P S O L S S O
(29358, 12)


In [63]:
# remove rows in val set 
remove_error(val_data)

index: 1458
text: I
slot label: O O
(3258, 12)


In [64]:
# remove rows in test set 
remove_error(test_data)

index: 1815
text:  ' '
slot label: O O O
index: 2164
text:  ' '
slot label: O O O
(3628, 12)


In [65]:
# check the proportion of intent labels in the dataset
print("train E:",train_data[train_data['Label']=='E']['Label'].count()/train_data.shape[0])
print("train I:",train_data[train_data['Label']=='I']['Label'].count()/train_data.shape[0])
print("train A:",train_data[train_data['Label']=='A']['Label'].count()/train_data.shape[0])
print("train O:",train_data[train_data['Label']=='O']['Label'].count()/train_data.shape[0])
print("train_data length:",train_data.shape[0])
print()
print("val E:",val_data[val_data['Label']=='E']['Label'].count()/val_data.shape[0])
print("val I:",val_data[val_data['Label']=='I']['Label'].count()/val_data.shape[0])
print("val A:",val_data[val_data['Label']=='A']['Label'].count()/val_data.shape[0])
print("val O:",val_data[val_data['Label']=='O']['Label'].count()/val_data.shape[0])
print("val_data length:",val_data.shape[0])
print()
print("test E:",test_data[test_data['Label']=='E']['Label'].count()/test_data.shape[0])
print("test I:",test_data[test_data['Label']=='I']['Label'].count()/test_data.shape[0])
print("test A:",test_data[test_data['Label']=='A']['Label'].count()/test_data.shape[0])
print("test O:",test_data[test_data['Label']=='O']['Label'].count()/test_data.shape[0])
print("test_data length:",test_data.shape[0])

train E: 0.06727297499829689
train I: 0.00991211935417944
train A: 0.02503576537911302
train O: 0.8977791402684107
train_data length: 29358

val E: 0.08011049723756906
val I: 0.008287292817679558
val A: 0.027317372621240024
val O: 0.8842848373235114
val_data length: 3258

test E: 0.06560088202866593
test I: 0.009095920617420067
test A: 0.027839029768467475
test O: 0.8974641675854466
test_data length: 3628


In [38]:
# create a function to convert the low dataset into jointBERT input
def create_input(data,set):
    seq_in = list(data['1'])
    with open('datasets/low/'+set+'/seq.in', 'w') as f:
        for item in seq_in:
            f.write("%s\n" % item)

    seq_out = list(data['10'])
    with open('datasets/low/'+set+'/seq.out', 'w') as f:
        for item in seq_out:
            f.write("%s\n" % item)

    label = list(data['Label'])
    with open('datasets/low/'+set+'/label', 'w') as f:
        for item in label:
            f.write("%s\n" % item)

In [39]:
# create the directory
!mkdir datasets/low/train
!mkdir datasets/low/val
!mkdir datasets/low/test

# create the input files for jointBERT
create_input(train_data,'train')
create_input(val_data,'val')
create_input(test_data,'test')

In [66]:
data = train_data
seq_in = list(data['1'])
seq_out = list(data['10'])
label = list(data['Label'])
seq_in = ['BOS ' + text + ' EOS\t' for text in seq_in]
result = list(zip(seq_in, seq_out))
result = ["".join(text) for text in result]
result_w_intent = list(zip(result, label))
result_w_intent = [" ".join(text) for text in result_w_intent]

with open('datasets/low/low.train.w-intent.iob', 'w') as f:
  for item in result_w_intent:
    f.write("%s\n" % item)

with open('datasets/low/low.train.iob', 'w') as f:
  for item in result:
    f.write("%s\n" % item)

In [67]:
data = val_data
seq_in = list(data['1'])
seq_out = list(data['10'])
label = list(data['Label'])
seq_in = ['BOS ' + text + ' EOS\t' for text in seq_in]
result = list(zip(seq_in, seq_out))
result = ["".join(text) for text in result]
result_w_intent = list(zip(result, label))
result_w_intent = [" ".join(text) for text in result_w_intent]

with open('datasets/low/low.dev.w-intent.iob', 'w') as f:
  for item in result_w_intent:
    f.write("%s\n" % item)

with open('datasets/low/low.dev.iob', 'w') as f:
  for item in result:
    f.write("%s\n" % item)

In [68]:
data = test_data
seq_in = list(data['1'])
seq_out = list(data['10'])
label = list(data['Label'])
seq_in = ['BOS ' + text + ' EOS\t' for text in seq_in]
result = list(zip(seq_in, seq_out))
result = ["".join(text) for text in result]
result_w_intent = list(zip(result, label))
result_w_intent = [" ".join(text) for text in result_w_intent]

with open('datasets/low/low.test.w-intent.iob', 'w') as f:
  for item in result_w_intent:
    f.write("%s\n" % item)

with open('datasets/low/low.test.iob', 'w') as f:
  for item in result:
    f.write("%s\n" % item)